# RC4
Es un algoritmo que se usa para cifrados de flujo, es utilizado dentro de protocolos de red populares como TSL y WEP para proteger el trafico de paquetes de una red

<img width="50%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/RC4.svg/1920px-RC4.svg.png">

# Algoritmos


## Operación XOR

In [236]:
from IPython.display import Math, Latex, display

def bin2(n): 
	return bin(n)[2:].zfill(8)

def showOp(a,b,c,d):
	display(Math(f'{a} \oplus {b} \oplus {c} = {d}'))
	display(Math(f'{bin2(a)} \\\\ {bin2(b)} \\\\ {bin2(c)} \\\\ \\rule{{1.5cm}}{{0.1pt}} \\\\ {bin2(d)} ' ))

k = b'a'[0]
m = b'c'[0]
i = 5
display(Math(f'k={k}={bin2(k)} \\\\ m={m}={bin2(m)} \\\\ i={i}={bin2(i)}'))

# Encrpytion
c = m ^ k ^ i 
showOp(m,k,i,c)

# Decryption
d = c ^ k ^ i 
showOp(c,k,i,d)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**Conlusion:**

La razón por la cual XOR es reversible no importa la cantidad de operandos se debe a la forma en que funciona la operación.

Por ejemplo, consideremos la operación XOR de tres bits a, b y c, donde d es el resultado: $d = a \oplus b \oplus c$. Si se conoce el valor de a, b y d, se puede encontrar el valor de c simplemente haciendo otra operación XOR: $c = a \oplus b \oplus d$

Este proceso se puede repetir para cualquier número de bits y cualquier número de operandos.

## Key-Scheduling Algorithm (KSA)
El KSA genera la permutación en el S-Box basándose en una clave de longitud variable ("keylength", entre 40 y 2048 bits).

Primero se llena el S-box (llamado "S" a continuación) con los valores de 0 a 255 en secuencia (permutación identidad); después se lo procesa 256 veces tal como lo hace el PRGA principal, excepto que, además, se mezclan bytes tomados de la clave ("K"), usando el siguiente pseudocódigo. 
~~~
for i from 0 to 255
    S[i] := i                                           (1)
endfor
j := 0                                                  (2)
for i from 0 to 255                                     (3)
    j := (j + S[i] + key[i mod keylength]) mod 256      (4)
    swap values of S[i] and S[j]                        (5)
endfor
~~~

In [257]:
def KSA(key):
    S = list(range(256)) # (1)
    j = 0   # (2)
    for i in range(256):  # (3)
        j = (j + S[i] + key[i % len(key)]) % 256 # (4) Mezcla los bytes con k
        S[i], S[j] = S[j], S[i] # (5) swap
    print("S-box:",S)
    return S

**Conslusión:** 
Podemos ver el KSA como una forma de ¿codificar? el *keystream* o llave

##  Pseudo-Random Generation Algorithm (PRGA)
~~~
i := 0
j := 0
while GeneratingOutput:
    i := (i + 1) mod 256
    j := (j + S[i]) mod 256
    swap values of S[i] and S[j]
    K := S[(S[i] + S[j]) mod 256]
    output K
endwhile
~~~
https://en.wikipedia.org/wiki/RC4


In [314]:
def PRGA(S,plaintext):
	i = j = 0
	ks = []
	for char in plaintext:
		i = (i + 1) % 256
		j = (j + S[i]) % 256
		S[i], S[j] = S[j], S[i]
		k = S[(S[i] + S[j]) % 256] # Pseudo-aleatoriedad
		ks.append(k)   
	print("Pseudo-aleatorizacin de la S-Box:",ks) 
	return ks

## Implemetación de RC4

En este caso se muestra que el algoritmo funciona para cualquier grupo de bytes

In [310]:
def rc4(key, plaintext):
  # Key-scheduling algorithm (KSA)
  S = KSA(key)
  # Pseudo-random generation algorithm (PRGA)    
  k = PRGA(S,plaintext)                                                                                                                                            
  # Operación XOR
  out = []
  for i in range(len(plaintext)):
    out.append(plaintext[i]^k[i])
    display(Math(f'\\text{{PRGA: }}{plaintext[i]} \oplus {k[i]}'))
  return bytes(out)

**Nota:** Cabe aclarar que la llave $k$ y el mensaje $m$ deben tener la misma longitud

In [324]:
import os
m = b'Hola'
k = os.urandom(len(m))  # Llave aleatoria de misma longitud del mensaje en bytes
print("Mensaje Original: ",m)
print("Llave: ",k)

Mensaje Original:  b'Hola'
Llave:  b'\xf8\x08\xbcI'


In [325]:
e = rc4(k,m)
print("Mensaje cifrado: ",e)

S-box: [119, 1, 49, 11, 12, 161, 17, 35, 80, 157, 250, 45, 74, 211, 41, 101, 253, 32, 31, 204, 40, 173, 215, 24, 34, 85, 155, 88, 150, 187, 43, 249, 162, 58, 39, 96, 209, 79, 148, 244, 201, 44, 23, 93, 97, 193, 168, 105, 75, 132, 110, 238, 111, 83, 154, 158, 206, 131, 236, 18, 165, 233, 227, 152, 67, 117, 71, 118, 50, 170, 108, 78, 13, 176, 103, 102, 169, 183, 141, 178, 109, 198, 100, 126, 216, 125, 104, 68, 115, 124, 222, 181, 160, 218, 52, 63, 172, 3, 120, 246, 232, 2, 225, 26, 235, 196, 5, 140, 73, 4, 114, 106, 76, 174, 156, 130, 8, 33, 163, 248, 7, 36, 194, 134, 208, 72, 27, 59, 87, 226, 185, 121, 231, 51, 127, 197, 128, 62, 55, 207, 142, 234, 167, 60, 219, 98, 195, 164, 81, 56, 237, 175, 107, 224, 6, 30, 37, 192, 151, 116, 77, 65, 189, 145, 123, 147, 220, 243, 171, 223, 255, 84, 90, 230, 191, 15, 190, 69, 177, 92, 242, 144, 138, 122, 64, 143, 159, 182, 221, 210, 53, 57, 229, 136, 29, 47, 14, 241, 146, 217, 9, 66, 254, 252, 19, 20, 247, 10, 135, 133, 48, 228, 203, 239, 99, 184, 245

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Mensaje cifrado:  b'y\x1bz\xbc'


In [326]:
c = rc4(k,e)
print("Mensaje descifrado: ",c)

S-box: [119, 1, 49, 11, 12, 161, 17, 35, 80, 157, 250, 45, 74, 211, 41, 101, 253, 32, 31, 204, 40, 173, 215, 24, 34, 85, 155, 88, 150, 187, 43, 249, 162, 58, 39, 96, 209, 79, 148, 244, 201, 44, 23, 93, 97, 193, 168, 105, 75, 132, 110, 238, 111, 83, 154, 158, 206, 131, 236, 18, 165, 233, 227, 152, 67, 117, 71, 118, 50, 170, 108, 78, 13, 176, 103, 102, 169, 183, 141, 178, 109, 198, 100, 126, 216, 125, 104, 68, 115, 124, 222, 181, 160, 218, 52, 63, 172, 3, 120, 246, 232, 2, 225, 26, 235, 196, 5, 140, 73, 4, 114, 106, 76, 174, 156, 130, 8, 33, 163, 248, 7, 36, 194, 134, 208, 72, 27, 59, 87, 226, 185, 121, 231, 51, 127, 197, 128, 62, 55, 207, 142, 234, 167, 60, 219, 98, 195, 164, 81, 56, 237, 175, 107, 224, 6, 30, 37, 192, 151, 116, 77, 65, 189, 145, 123, 147, 220, 243, 171, 223, 255, 84, 90, 230, 191, 15, 190, 69, 177, 92, 242, 144, 138, 122, 64, 143, 159, 182, 221, 210, 53, 57, 229, 136, 29, 47, 14, 241, 146, 217, 9, 66, 254, 252, 19, 20, 247, 10, 135, 133, 48, 228, 203, 239, 99, 184, 245

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Mensaje descifrado:  b'Hola'


# IVs con RC4

* We increment i and use it to index into S, generating a new value S[i].
* We increment j by S[i] modulo 256, and use it to index into S, generating a new value S[j].
* We swap S[i] and S[j].
* We XOR the current byte of data `data[k]` with the byte at index (S[i] + S[j]) % 256 in S, and with the byte at index k % len(iv) in iv.
* We append the resulting byte to our output array out.

In [323]:
from typing import List

def rc4_2(key: bytes, plaintext: bytes, iv: bytes) -> bytes:
    
    # Key-Scheduling Algorithm (KSA)
    S = KSA(key)

    k = PRGA(S,plaintext)

    # Operación XOR
    out = []
    for i in range(len(plaintext)):
        iv_i = iv[i % len(iv)]
        out.append(plaintext[i]^k[i] ^ iv_i)
        display(Math(f'\\text{{PRGA: }}{plaintext[i]} \oplus {k[i]} \oplus {iv_i}'))
        
    return bytes(out)
    

In [320]:
import os
import random
iv = [random.randint(0, 255) for i in range(4)]
m = b'Hola'
k = os.urandom(len(m))  # Llave aleatoria de misma longitud del mensaje en bytes
print(k)
print("Mensaje Original: ",m)
print("Llave: ",k)
print("IV: ",iv)

b'\xc3m\xfe\x9d'
Mensaje Original:  b'Hola'
Llave:  b'\xc3m\xfe\x9d'
IV:  [222, 200, 60, 120]


In [321]:
e = rc4_2(k,m,iv)
print("Mensaje cifrado: ",e)

S-box: [195, 21, 31, 209, 152, 37, 14, 178, 125, 128, 238, 158, 109, 231, 235, 151, 28, 184, 248, 15, 127, 174, 185, 63, 92, 42, 123, 186, 217, 99, 177, 59, 65, 182, 204, 140, 52, 10, 218, 118, 222, 107, 94, 74, 175, 51, 255, 203, 190, 211, 205, 2, 173, 198, 78, 90, 23, 61, 154, 43, 77, 46, 86, 45, 97, 137, 224, 192, 199, 50, 189, 161, 147, 44, 139, 180, 221, 85, 75, 116, 38, 121, 149, 133, 156, 81, 207, 73, 8, 206, 143, 80, 167, 60, 82, 254, 67, 131, 171, 253, 102, 4, 165, 202, 83, 30, 141, 153, 105, 181, 58, 241, 162, 96, 226, 138, 3, 210, 120, 117, 68, 196, 100, 34, 227, 113, 70, 53, 243, 247, 200, 142, 19, 169, 236, 40, 145, 13, 134, 124, 110, 214, 66, 89, 191, 245, 16, 33, 159, 71, 129, 111, 49, 135, 163, 219, 114, 48, 47, 56, 157, 130, 229, 132, 223, 79, 146, 166, 35, 91, 20, 197, 64, 115, 26, 57, 252, 103, 194, 212, 36, 18, 172, 179, 240, 213, 55, 41, 95, 0, 122, 136, 126, 176, 72, 84, 9, 220, 22, 5, 54, 193, 88, 168, 12, 93, 234, 87, 233, 188, 183, 201, 62, 29, 225, 150, 228, 1

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Mensaje cifrado:  b'\xc2\xab8/'


In [322]:
c = rc4_2(k,e,iv)
print("Mensaje descifrado: ",c)

S-box: [195, 21, 31, 209, 152, 37, 14, 178, 125, 128, 238, 158, 109, 231, 235, 151, 28, 184, 248, 15, 127, 174, 185, 63, 92, 42, 123, 186, 217, 99, 177, 59, 65, 182, 204, 140, 52, 10, 218, 118, 222, 107, 94, 74, 175, 51, 255, 203, 190, 211, 205, 2, 173, 198, 78, 90, 23, 61, 154, 43, 77, 46, 86, 45, 97, 137, 224, 192, 199, 50, 189, 161, 147, 44, 139, 180, 221, 85, 75, 116, 38, 121, 149, 133, 156, 81, 207, 73, 8, 206, 143, 80, 167, 60, 82, 254, 67, 131, 171, 253, 102, 4, 165, 202, 83, 30, 141, 153, 105, 181, 58, 241, 162, 96, 226, 138, 3, 210, 120, 117, 68, 196, 100, 34, 227, 113, 70, 53, 243, 247, 200, 142, 19, 169, 236, 40, 145, 13, 134, 124, 110, 214, 66, 89, 191, 245, 16, 33, 159, 71, 129, 111, 49, 135, 163, 219, 114, 48, 47, 56, 157, 130, 229, 132, 223, 79, 146, 166, 35, 91, 20, 197, 64, 115, 26, 57, 252, 103, 194, 212, 36, 18, 172, 179, 240, 213, 55, 41, 95, 0, 122, 136, 126, 176, 72, 84, 9, 220, 22, 5, 54, 193, 88, 168, 12, 93, 234, 87, 233, 188, 183, 201, 62, 29, 225, 150, 228, 1

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Mensaje descifrado:  b'Hola'


# Q&A
* ¿Todos los cifrados simétricos utilizan el mismo metodo para cifrar y descifrar?